In [0]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

!pip install pmdarima
!pip install statsmodels --upgrade

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
df = pd.read_csv('hospitalizados.csv',encoding = "ISO-8859-1", index_col=0, header=None).T  # Lectura del fichero de entrada y transpuesta del mismo
df = df.rename(columns={'Fecha': 'Date'}) # Ajuste del nombre de las columnas
df['Date']= pd.to_datetime(df['Date']) # Conversión a tipo fecha
df.set_index('Date', inplace=True) # La columna Fechas como índice
df=df.apply(pd.to_numeric) # Hacer que los tipos de datos sean numéricos
df = df.sort_values('Date') # Ordenar en función de la fecha
df=df.interpolate('zero', fill_value=0, limit_direction='backward') # A 0 los primeros valores antes de un número
df=df.interpolate(method='linear', axis=0).ffill().bfill() # Valores intermedios los interpolas
df

Vamos a analizar el hospital de Tomelloso

In [0]:
df = df['H. Tomelloso']

# **MODELO ARIMA**
Ref: https://www.machinelearningplus.com/time-series/arima-model-time-series-forecasting-python/<br/>
**Auto Regressive Integrated Moving Average**:<br/>Se trata de un algoritmo de predicción basada en la idea de que situaciones pasadas pueden ser usadas para predecir el futuro.<br/>
Cualquier serie temporal *no estacional* que tenga un patron y que no tenga ruido blanco aleatorio puede ser modelado con ARIMA.<br/>
Los hiperparámetros de ARIMA son 3:


*   **p**: orden del término AR.
*   **q**: orden del término MA.
*   **d**: diferencia necesaria para hacer las series estacionarias.

Si se tratase de una serie temporal estacionaria, deberiamos emplear *SARIMA*.

Comprobar si una serie es estacionaria: para comprobarlo la forma es restar el valor previo al actual, de esta manera sacamos d, que será el número mínimo de diferencia necesaria para que la serie sea estacionaria. Si ya de por si la serie es estacionaria, **d** = 0.<br/>
**p** se refiere al número de retraso de Y para ser usado como predictor.<br/>
**q** se refiere al número de errores de pronóstico retardado.<br/>

In [0]:
from statsmodels.tsa.stattools import adfuller
from numpy import log
result = adfuller(df)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])

Como el p-valor es superior al nivel de significacion, vamos a diferencias las series y ver la figura de autocorrelación




In [0]:
import numpy as np, pandas as pd
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.figsize':(9,7), 'figure.dpi':120})
fig, axes = plt.subplots(3, 2, sharex=True)

# Original Series
axes[0, 0].plot(df.values); axes[0, 0].set_title('Original Series')
plot_acf(df.values, ax=axes[0, 1])

# 1st Differencing
axes[1, 0].plot(np.diff(df)); axes[1, 0].set_title('1st Order Differencing')
plot_acf(np.diff(df), ax=axes[1, 1])

# 2nd Differencing
axes[2, 0].plot(np.diff(np.diff(df))); axes[2, 0].set_title('2nd Order Differencing')
plot_acf(np.diff(np.diff(df)), ax=axes[2, 1])

plt.show()

Encontrar el valor de **p**.<br/>
Vamos a identificar si el modelo necesita términos AR. Para ello inspeccionaremos el PARTIAL Autocorrelation (PACF) plot, que lo que hace es la correlación entre las series y su retardo, excluyendo las contribuciones de los retrasos intermedios.



In [0]:
# PACF plot of 1st differenced series
plt.rcParams.update({'figure.figsize':(9,3), 'figure.dpi':120})

fig, axes = plt.subplots(1, 2, sharex=True)
axes[0].plot(np.diff(df)); axes[0].set_title('1st Differencing')
axes[1].set(ylim=(0,5))
plot_pacf(np.diff(df), ax=axes[1])

plt.show()

Encontrar el valor de **q**.<br/>
Nos centraremos en el ACF plot para saber el número de términos MA, siéndo éste el error del retraso en la predicción.


In [0]:
plt.rcParams.update({'figure.figsize':(9,3), 'figure.dpi':120})
fig, axes = plt.subplots(1, 2, sharex=True)

axes[0].plot(np.diff(df)); axes[0].set_title('1st Differencing')
axes[1].set(ylim=(0,1.2))
plot_acf(np.diff(df), ax=axes[1])

plt.show()

Construcción del modelo ARIMA:<br/>
El propio modelo nos muestra los coeficientes y los pesos que les ha dado a dichos coeficientes.
Para que un modelo sea bueno, el término MA2 debe ser cercano a 0 y el P>|z| debería ser idealmente menor que 0.05.<br/>
debemos buscar tambien reducir el AIC.<br/>
Ref: https://www.statsmodels.org/stable/generated/statsmodels.tsa.arima_model.ARIMA.html

In [0]:
from statsmodels.tsa.arima_model import ARIMA

# 1,1,2 ARIMA Model
model = ARIMA(df.values, order=(1,1,1))
model_fit = model.fit(disp=0)
print(model_fit.summary())

Vamos a observar los residuos para ver que no hay patrones, buscando *medias* y *varianzas* constantes.<br/>
Si en el error residual, la media es cercana a 0, y una varianza uniforme, iremos por buen camino.

In [0]:
# Plot residual errors
residuals = pd.DataFrame(model_fit.resid)
fig, ax = plt.subplots(1,2)
residuals.plot(title="Residuals", ax=ax[0])
residuals.plot(kind='kde', title='Density', ax=ax[1])
plt.show()

Comprobación de lo predicho respecto a lo real.<br/>
Al añadir dynamic=False, los valores de retardo de la muestra son empleados para la predicción

In [0]:
# Actual vs Fitted
model_fit.plot_predict(dynamic=False)
plt.show()

Comprobación del modelo manualmente empleando conjunto de test y conjunto de entrenamiento.<br/>
Partimos el dataset en entrenamiento y test.

In [0]:
# Comprobamos la longitud de nuestro dataset
print(df.count) #74

from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2,shuffle=False)

In [0]:
# Build Model
model = ARIMA(train, order=(3,2,1))  
#model = ARIMA(train, order=(1, 1, 1))  
fitted = model.fit(disp=-1)  
print(fitted.summary())

# Forecast
fc, se, conf = fitted.forecast(len(test.index), alpha=0.05)  # 95% conf

# Make as pandas series
fc_series = pd.Series(fc, index=test.index)
lower_series = pd.Series(conf[:, 0], index=test.index)
upper_series = pd.Series(conf[:, 1], index=test.index)

# Plot
plt.figure(figsize=(12,5), dpi=100)
plt.plot(train, label='training')
plt.plot(test, label='actual')
plt.plot(fc_series, label='forecast')
plt.fill_between(lower_series.index, lower_series, upper_series, 
                 color='k', alpha=.15)
plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()

Métricas para las predicciones de series temporales:
1.   Mean Absolute Percentage Error (MAPE)
2.   Mean Error (ME)
3.   Mean Absolute Error (MAE)
4.   Mean Percentage Error (MPE)
5.   Root Mean Squared Error (RMSE)
6.   Lag 1 Autocorrelation of Error (ACF1)
7.   Correlation between the Actual and the Forecast (corr)
8.   Min-Max Error (minmax)

Básicamente se emplean MAPE, Correlation y Min-Max, esto es debido a que estas 3 nos dan valores entre 0 y 1, mientras que las otras son medidas cuantitativas con su escala propia.

In [0]:
from statsmodels.tsa.stattools import acf
# Accuracy metrics
def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    mins = np.amin(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    maxs = np.amax(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    minmax = 1 - np.mean(mins/maxs)             # minmax
    acf1 = acf(fc-test)[1]                      # ACF1
    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse, 'acf1':acf1, 
            'corr':corr, 'minmax':minmax})

forecast_accuracy(fc, test.values)

El mape, el valor que nos salga, nos indica la imprecisión del modelo, por lo que si hacemos 1-MAPE, obtendremos la precisión de nuestro modelo en el número de predicciones indicadas.

# Auto ARIMA

In [0]:
#!pip install pmdarima
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm

model = pm.auto_arima(df, start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

print(model.summary())

Interpretación de los plots residuales en ARIMA

In [0]:
#!pip install statsmodels --upgrade
model.plot_diagnostics(figsize=(7,5))
plt.show()

*   Arriba izda: errores residuales fluctuan respecto a media 0 y tienen una varianza uniforme.
*   Arriba drcha: distribución normal media cero
*   Abajo izda: si los puntos están alineados con la linea roja
*   Abajo drcha: si en el correlograma apareciese algún tipo de patrón



# Predicción de valores

In [0]:
import datetime
# Forecast
n_periods = 24
fc, confint = model.predict(n_periods=n_periods, return_conf_int=True)

#index_of_fc= pd.date_range(start=(df.index[-1]+datetime.timedelta(days=1)), periods=n_periods, freq='D')
index_of_fc= pd.date_range(start=(df.index[-1]), periods=n_periods, freq='D')

print(index_of_fc)
# make series for plotting purpose
fc_series = pd.Series(fc, index=index_of_fc)
lower_series = pd.Series(confint[:, 0], index=index_of_fc)
upper_series = pd.Series(confint[:, 1], index=index_of_fc)

# Plot
plt.plot(df)
plt.plot(fc_series, color='darkgreen')
plt.fill_between(lower_series.index, 
                 lower_series, 
                 upper_series, 
                 color='k', alpha=.15)

plt.title("Final Forecast")
plt.ylim(df.min(), df.max())
plt.xticks(rotation=90)
!mkdir img/
plt.savefig('img/hospitalizados_cm.jpg', bbox_inches='tight')
plt.show()



In [0]:
fc_series

Exportamos los resultados a un excel

In [0]:
!mkdir rslt/
df.to_csv('rslt/fc_series.csv', float_format='%.3f', decimal=",", sep = ';', encoding='ISO-8859-1')